# Model Time Series ML to predict metro frequentation during JO period with JO of stations rattached to 2 sites without known past events by applicating coefficients found on sites with known past events

In [31]:
station = "CHAMP DE MARS-TOUR EIFFEL"
site = ["ARENA CHAMP DE MARS","STADE TOUR EIFFEL"]

In [32]:
import pandas as pd
import numpy as np

In [33]:
def events(site):
    if site == "ROLAND-GARROS":
        return "EVENT_RG"
    elif site == "PARC DES PRINCES":
        return "EVENT_PDP"
    elif site == "STADE DE FRANCE":
        return "EVENT_SDF"
    elif site == "GRAND PALAIS":
        return "EVENT_GP"
    elif site == "ARENA INVALIDES":
        return "EVENT_INV"
    elif site == "STADE DE LA CONCORDE":
        return "EVENT_CCD"
    elif site == "ARENA CHAMP DE MARS":
        return "EVENT_CDM"
    elif site == "STADE TOUR EIFFEL":
        return "EVENT_TEF"
    elif site == "ARENA PARIS SUD":
        return "EVENT_SUD"
    elif site == "ARENA LA CHAPELLE":
        return "EVENT_CHP"
    elif site == "ARENA BERCY":
        return "EVENT_BER"
    else:
        return 0

In [34]:
data=pd.read_csv(f"98-Forecast/forecast_ssjo_{station}.csv")
data.head()

,ds,yhat,station
0,2024-07-01,8.097728,CHAMP DE MARS-TOUR EIFFEL
1,2024-07-02,8.074856,CHAMP DE MARS-TOUR EIFFEL
2,2024-07-03,8.049079,CHAMP DE MARS-TOUR EIFFEL
3,2024-07-04,8.140618,CHAMP DE MARS-TOUR EIFFEL
4,2024-07-05,8.130671,CHAMP DE MARS-TOUR EIFFEL


In [35]:
geoloc=pd.read_csv("99-Data_Clean/geoloc_jo24.csv")
geoloc.head()

,lieu,type,lat,lng,capacite
0,STADE DE FRANCE-SAINT-DENIS,station,48.917812,2.351067,19636.0
1,SAINT-DENIS-PORTE DE PARIS,station,48.929499,2.356273,13758.0
2,LA PLAINE-STADE DE FRANCE,station,48.918107,2.362326,22415.0
3,BOULEVARD VICTOR,station,48.839364,2.270485,4430.0
4,PORTE D'AUTEUIL,station,48.847677,2.257985,1968.0


In [36]:
df_coeff = pd.read_csv("99-Data_Clean/coeff_stations_ssevent.csv")
df_coeff.head()

,lieu,site model,station model,EVENT,EVENT_LOG
0,MUSEE D'ORSAY,ROLAND-GARROS,PORTE D'AUTEUIL,-11.520813,1.169022
1,SEGUR,ROLAND-GARROS,PORTE D'AUTEUIL,-11.520813,1.169022
2,PONT DE L'ALMA,ROLAND-GARROS,PORTE D'AUTEUIL,-11.520813,1.169022
3,CHAMP DE MARS-TOUR EIFFEL,ROLAND-GARROS,PORTE D'AUTEUIL,-11.520813,1.169022
4,PORTE DE LA CHAPELLE,ROLAND-GARROS,PORTE DE SAINT-CLOUD,1.285228,-0.113329


In [37]:
event=pd.read_csv("99-Data_Clean/event_jo.csv")
event.head()

,ds,EVENT_PDP,EVENT_SDF,EVENT_GP,EVENT_INV,EVENT_CCD,EVENT_YDM,EVENT_CDM,EVENT_TEF,EVENT_RG,EVENT_SUD,EVENT_CHP,EVENT_BER
0,2024-07-01,0,0,0,0,0,0,0,0,0,0,0,0
1,2024-07-02,0,0,0,0,0,0,0,0,0,0,0,0
2,2024-07-03,0,0,0,0,0,0,0,0,0,0,0,0
3,2024-07-04,0,0,0,0,0,0,0,0,0,0,0,0
4,2024-07-05,0,0,0,0,0,0,0,0,0,0,0,0


In [38]:
event[['ds', events(site[0]), events(site[1])]]

,ds,EVENT_CDM,EVENT_TEF
0,2024-07-01,0,0
1,2024-07-02,0,0
2,2024-07-03,0,0
3,2024-07-04,0,0
4,2024-07-05,0,0
...,...,...,...
57,2024-08-27,0,0
58,2024-08-28,0,0
59,2024-08-29,0,0
60,2024-08-30,0,0


In [39]:
data=data.merge(event[['ds', events(site[0]), events(site[1])]],on="ds")

In [40]:
cap1 = 0
cap2 = 0
for i in range(len(geoloc['lieu'])):
    if site[0] == geoloc['lieu'][i]:
        cap1 = geoloc['capacite'][i]
    else:
        pass
for i in range(len(geoloc['lieu'])):
    if site[1] == geoloc['lieu'][i]:
        cap2 = geoloc['capacite'][i]
    else:
        pass

In [41]:
print(cap1)
print(cap2)

8356.0
12860.0


In [42]:
data['cap1']=data[events(site[0])].apply(lambda x:1 if x==0 else cap1)
data['cap2']=data[events(site[1])].apply(lambda x:1 if x==0 else cap2)
data.head()

,ds,yhat,station,EVENT_CDM,EVENT_TEF,cap1,cap2
0,2024-07-01,8.097728,CHAMP DE MARS-TOUR EIFFEL,0,0,1.0,1.0
1,2024-07-02,8.074856,CHAMP DE MARS-TOUR EIFFEL,0,0,1.0,1.0
2,2024-07-03,8.049079,CHAMP DE MARS-TOUR EIFFEL,0,0,1.0,1.0
3,2024-07-04,8.140618,CHAMP DE MARS-TOUR EIFFEL,0,0,1.0,1.0
4,2024-07-05,8.130671,CHAMP DE MARS-TOUR EIFFEL,0,0,1.0,1.0


In [43]:
coeff_1 = float(df_coeff[(df_coeff['lieu'] == station)]['EVENT'].values)
coeff_2 = float(df_coeff[(df_coeff['lieu'] == station)]['EVENT_LOG'].values)

In [44]:
data['yhatjo']=data['yhat']+(coeff_1*data[events(site[0])])+(coeff_2*np.log(data['cap1']))+(coeff_1*data[events(site[1])])+(coeff_2*np.log(data['cap2']))

In [45]:
data[25:40]

,ds,yhat,station,EVENT_CDM,EVENT_TEF,cap1,cap2,yhatjo
25,2024-07-26,8.269752,CHAMP DE MARS-TOUR EIFFEL,0,0,1.0,1.0,8.269752
26,2024-07-27,7.845388,CHAMP DE MARS-TOUR EIFFEL,1,1,8356.0,12860.0,6.422041
27,2024-07-28,7.611109,CHAMP DE MARS-TOUR EIFFEL,1,1,8356.0,12860.0,6.187762
28,2024-07-29,8.214869,CHAMP DE MARS-TOUR EIFFEL,1,1,8356.0,12860.0,6.791522
29,2024-07-30,8.182116,CHAMP DE MARS-TOUR EIFFEL,1,1,8356.0,12860.0,6.758768
30,2024-07-31,8.143092,CHAMP DE MARS-TOUR EIFFEL,1,1,8356.0,12860.0,6.719744
31,2024-08-01,8.217932,CHAMP DE MARS-TOUR EIFFEL,1,1,8356.0,12860.0,6.794585
32,2024-08-02,8.187822,CHAMP DE MARS-TOUR EIFFEL,1,1,8356.0,12860.0,6.764475
33,2024-08-03,7.744394,CHAMP DE MARS-TOUR EIFFEL,1,1,8356.0,12860.0,6.321047
34,2024-08-04,7.491580,CHAMP DE MARS-TOUR EIFFEL,0,1,1.0,12860.0,7.031914


In [46]:
data_fin = pd.concat([data['ds'],data['yhatjo'],data['station']], axis=1, ignore_index=True)
data_fin.columns = ['ds', 'yhat', 'station']
data_fin

,ds,yhat,station
0,2024-07-01,8.097728,CHAMP DE MARS-TOUR EIFFEL
1,2024-07-02,8.074856,CHAMP DE MARS-TOUR EIFFEL
2,2024-07-03,8.049079,CHAMP DE MARS-TOUR EIFFEL
3,2024-07-04,8.140618,CHAMP DE MARS-TOUR EIFFEL
4,2024-07-05,8.130671,CHAMP DE MARS-TOUR EIFFEL
...,...,...,...
57,2024-08-27,7.776492,CHAMP DE MARS-TOUR EIFFEL
58,2024-08-28,7.772692,CHAMP DE MARS-TOUR EIFFEL
59,2024-08-29,7.887403,CHAMP DE MARS-TOUR EIFFEL
60,2024-08-30,7.901208,CHAMP DE MARS-TOUR EIFFEL


In [24]:
data_fin.to_csv(f"98-Forecast/forecast_jo_{station}.csv", index=False)